In [2]:

import re
from pathlib import Path
from typing import List, Tuple


from langchain.document_loaders import CSVLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_ollama import OllamaLLM
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_core.documents import Document
from langchain_community.vectorstores import FAISS
import ollama

from pathlib import Path
import pdf2image
try:
    from PIL import Image
except ImportError:
    import Image
import pytesseract
pytesseract.pytesseract.tesseract_cmd = r"C:\Users\mathe\repos\Wine-Sommelier-AI\Tesseract-OCR\tesseract.exe"


In [13]:
data_folder = Path("data")
pdf_files = list(data_folder.glob("*.pdf"))
csv_files = list(data_folder.glob("*.csv"))
pages_description = []
pages_matches = []


In [14]:

def pdf_to_img(pdf_file):
    return pdf2image.convert_from_path(pdf_file, poppler_path=r"C:\Users\mathe\repos\local\vinho\poppler\poppler-24.08.0\Library\bin")

def ocr_core(file):
    return pytesseract.image_to_string(file)


In [16]:
for pdf_file in pdf_files:
    images = pdf_to_img(str(pdf_file))
    for pg, img in enumerate(images):
        text = ocr_core(img)
        if text.strip():
            pages_description.append(Document(page_content=text, metadata={"source": str(pdf_file), "page": pg + 1}))


In [15]:
for csv_file in csv_files:
    loader = CSVLoader(str(csv_file), encoding="utf-8")
    csv_pages = loader.load()
    pages_matches.extend(csv_pages)


In [17]:
def clean_text(texto: str) -> str:
    texto = re.sub(r"(?<!\n)\n(?!\n)", " ", texto)
    return texto.strip()


In [18]:
for page in pages_description:
    page.page_content = clean_text(page.page_content)

for page in pages_matches:
    page.page_content = clean_text(page.page_content)


In [19]:
print(pages_description[90].page_content)


Vinhos e Pratos

Apresentamos neste capitulo uma avaliacaéo sucinta de um conjunto de vinhos bastante populares, e a cada um deles relacionamos alguns dos pratos com os quais eles harmonizam-se bem.

As harmonizagoes sugeridas consideram apenas os ele- mentos mais marcantes da comida, como acidez, textura e gorduras, e os do vinho, como corpo, tanicidade e aci- dez, e buscam preservar o equilibrio das partes. Outros componentes importantes, como a natureza do molho e a presencga de condimentos, nao foram considerados.

Fica entao a ressalva de que, diante de um prato es- pecifico, a influéncia de todos os componentes envolvidos no processo de harmonizacao precisaria ser considerada, e a recomendacao de casamento pode ter de ser alterada em certos casos.

69


In [21]:

docs_description = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200).split_documents(pages_description)
docs_matches = RecursiveCharacterTextSplitter(chunk_size=5000, chunk_overlap=200).split_documents(pages_matches)



In [22]:
len(docs_description), len(docs_matches)


(356, 122)

In [23]:
docs_matches[10].page_content


'\ufeffname: Vinho Rosé Góes Tempos Pétalas Cabernet Franc 2023 short_description: Delicados aromas de frutas vermelhas e toques florais, perfeito para massas, risotos e carnes brancas. pix_total: 62.4 image_src: https://cdn.dooca.store/158171/products/dne7b7kp0yy0dggetftbwuegpfjjz4kbonzf.jpg?v=1722640506'

In [24]:
docs_description[91].page_content


'Resumo Executivo\n\nCom o objetivo de simplificar o processo de harmoniza- cao, momentaneamente, ignoraremos a influéncia dos aromas, sabores, buqué, teor alcodlico, e tomaremos como elemento decisério para o casamento de vinhos e comidas, de um lado, a intensidade dos taninos, o grau de acidez e a dosagem de agtcar dos vinhos e, de ou- tro, as caracteristicas marcantes da comida categoriza- das como dcida, salgada, amarga, apimentada, gordu- rosa, 10dada e proteica.\n\nConsiderando apenas esses elementos, 0 processo de har- monizacao é resumido pelo grafo da Fig. 1, no qual é interessante observar que a carne vermelha ao propiciar a mistura, na boca, de suas proteinas com os taninos, torna estes mais macios, e assim, reduz a aspereza do vinho tanico. Ou seja, a carne melhora o vinho. O grafo também destaca que a acidez do vinho casa per- feitamente com quase todos os tipos de comidas, excetu- ando-se apenas a comida amarga e a iodada.\n\n39'

In [25]:
for doc in docs_description:
    doc.page_content = "passage: " + doc.page_content
    
for doc in docs_matches:
    doc.page_content = "passage: " + doc.page_content
    


In [26]:
docs_description[91].page_content


'passage: Resumo Executivo\n\nCom o objetivo de simplificar o processo de harmoniza- cao, momentaneamente, ignoraremos a influéncia dos aromas, sabores, buqué, teor alcodlico, e tomaremos como elemento decisério para o casamento de vinhos e comidas, de um lado, a intensidade dos taninos, o grau de acidez e a dosagem de agtcar dos vinhos e, de ou- tro, as caracteristicas marcantes da comida categoriza- das como dcida, salgada, amarga, apimentada, gordu- rosa, 10dada e proteica.\n\nConsiderando apenas esses elementos, 0 processo de har- monizacao é resumido pelo grafo da Fig. 1, no qual é interessante observar que a carne vermelha ao propiciar a mistura, na boca, de suas proteinas com os taninos, torna estes mais macios, e assim, reduz a aspereza do vinho tanico. Ou seja, a carne melhora o vinho. O grafo também destaca que a acidez do vinho casa per- feitamente com quase todos os tipos de comidas, excetu- ando-se apenas a comida amarga e a iodada.\n\n39'

In [27]:
embedding = HuggingFaceEmbeddings(model_name="intfloat/multilingual-e5-base")


c:\Users\mathe\repos\Wine-Sommelier-AI\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [28]:
db_description = FAISS.from_documents(docs_description, embedding)
db_matches = FAISS.from_documents(docs_matches, embedding)

db_description.save_local("./faiss")
db_matches.save_local("./faiss")


In [29]:
db_description.save_local("./faiss/description")
db_matches.save_local("./faiss/matches")


In [ ]:
MODEL = "mistral:instruct"


In [ ]:
ollama.pull(MODEL)


ProgressResponse(status='success', completed=None, total=None, digest=None)

In [ ]:
llm = OllamaLLM(model=MODEL)


In [ ]:
def get_answer(question: str) -> str:
    return llm.invoke(question)


In [ ]:
# db = FAISS.load_local("faiss", embedding, allow_dangerous_deserialization=True)
retriever_description = db_description.as_retriever(search_kwargs={"k": 3})
retriever_matches = db_matches.as_retriever(search_kwargs={"k": 3})


In [ ]:
def formatar_resposta(texto):
    texto = texto.replace(". ", ".\n")
    texto = texto.replace(" -", "\n-")
    return texto.strip()


# Descricao


In [ ]:
pergunta = "Qual vinho melhor orna com Boeuf Bourguignon?"


In [ ]:
role = """Você é um sommelier experiente.
          Sua função é recomendar vinhos assertivos ao cliente
          voce acabou de receber um prato, e deve usar apenas o contexto fornecido para a tarefa,
          descreva brevemente como deveria ser o vinho ideal para esse prato,
          falando sobre o tipo de vinho, cor, aroma, corpo, acidez, etc,
          seja sucinto e em poucas palavras
          exemplo: 
          "Vinho Branco, acididade leve e aromas frutados que harmonizam bem com o sabor cremoso do queijo"
          "Vinho Verde, fresco e frutado, combina bem com a doçura e acidez da morango. 
          "vinho branco leve, com aromas citrus ou frutados, de corpo médio, para que a saborosa carne do camarão se destaque."
          """


In [ ]:
# role = """
# Você é um sommelier especialista em vinhos brasileiros, com conhecimento profundo sobre harmonização gastronômica. Sua função é fornecer recomendações precisas e educativas sobre vinhos, baseando-se nas informações recuperadas do banco de dados de vinhos.

# Restrições:
# - Baseie-se apenas nas informações recuperadas do banco de dados
# - Não invente características de vinhos não presentes nos dados
# - Se não houver informação suficiente, seja honesto sobre as limitações
# - Priorize vinhos brasileiros da base de dados
# - Mantenha recomendações dentro do contexto dos vinhos disponíveis
# """


In [ ]:
query = "query: " + pergunta
docs_relevantes = retriever_description.get_relevant_documents(query)


docs_relevantes


[Document(id='fc71e849-c9ea-48af-a2d1-e2efecb8f55f', metadata={'source': 'data\\MANUAL-CARTAS-DE-VINHO_2024-ADREPES.pdf', 'page': 80}, page_content='passage: Embora os vinhos tintos sejam geralmente os mais competentes, alguns vinhos brancos entroncados de corpo, conseguem lidar com a exigéncia dos sabores do prato e enfrentar a substancia e firmeza da carne, assim como a envolvéncia do caldo. A incluséo de alguns elementos que acrescentem frescura, tal como citricos e ervas aromaticas, podem facilitar a vida ao vinho branco. Possuir acidez intensa equilibra a consisténcia do prato. Vinhos com fermentagao e maturagaéo em madeira conferem na boca uma sensagao mais rica e aveludada que apetrecha o vinho para uma harmonia bem sucedida.\n\nVinho de corpo médio a encorpado, boa carga de taninos, mas polidos e sem adstringéncia, de paladar cheio e envolvente, com pujanga frutada e complexidade. O corpo médio a cheio do vinho equipara a riqueza do prato. O paladar abundante de fruta complemen

In [ ]:
contexto = ". ".join([doc.page_content for doc in docs_relevantes])


In [ ]:
print(formatar_resposta(contexto))


passage: Embora os vinhos tintos sejam geralmente os mais competentes, alguns vinhos brancos entroncados de corpo, conseguem lidar com a exigéncia dos sabores do prato e enfrentar a substancia e firmeza da carne, assim como a envolvéncia do caldo.
A incluséo de alguns elementos que acrescentem frescura, tal como citricos e ervas aromaticas, podem facilitar a vida ao vinho branco.
Possuir acidez intensa equilibra a consisténcia do prato.
Vinhos com fermentagao e maturagaéo em madeira conferem na boca uma sensagao mais rica e aveludada que apetrecha o vinho para uma harmonia bem sucedida.

Vinho de corpo médio a encorpado, boa carga de taninos, mas polidos e sem adstringéncia, de paladar cheio e envolvente, com pujanga frutada e complexidade.
O corpo médio a cheio do vinho equipara a riqueza do prato.
O paladar abundante de fruta complementa a suculéncia, a tenrura da carne e sabor do caldo.
Os taninos e acidez trazem balango e frescura ao paladar, tal como.
potenciam a persisténcia de s

In [ ]:
prompt = f"""
<role>
{role}
</role>
<contexto>:
{contexto}
</contexto>
<pergunta>:
{pergunta}
</pergunta>
<resposta>:
...
"""


In [ ]:
description = get_answer(prompt)


In [ ]:
description


' Vinho tinto bem encorpado, como o Cabernet Sauvignon.'

In [ ]:

role = """Você é um sommelier experiente.
          trabalha em um renomado restaurante 5 estrelas
          Sua função é recomendar vinhos assertivos ao cliente
          voce acabou de receber uma descrição de vinho, e deve usar o contexto para construir uma recomendação apropriada,
          lembre que voce esta falando diretamente com o cliente, entao seja educado e profissional,
          tambem explique o porque o vinho escolhido é o melhor para o cliente, e como ele orna com o prato"""


In [ ]:
query = "query: " + description
docs_relevantes = retriever_matches.get_relevant_documents(query)

docs_relevantes


[Document(id='e3913fb6-9f86-4e7b-be56-0f8d84417238', metadata={'source': 'data\\vinhos_completos.csv', 'row': 59}, page_content='passage: \ufeffname: Vinho Tinto Simis Cabernet Sauvignon Gran Reserva 2019 short_description: Cabernet Sauvignon robusto e complexo, com notas de cereja, figo e tabaco, ideal para carnes intensas e queijos fortes. pix_total: 130.56 image_src: https://cdn.dooca.store/158171/products/qo1mrdylixddysxzn2t3hvrvstrxmnzlcwry.jpg?v=1722640536'),
 Document(id='10abb3b1-fd1c-4c20-bcc3-efac684b41ec', metadata={'source': 'data\\vinhos_completos.csv', 'row': 59}, page_content='passage: \ufeffname: Vinho Tinto Simis Cabernet Sauvignon Gran Reserva 2019 short_description: Cabernet Sauvignon robusto e complexo, com notas de cereja, figo e tabaco, ideal para carnes intensas e queijos fortes. pix_total: 130.56 image_src: https://cdn.dooca.store/158171/products/qo1mrdylixddysxzn2t3hvrvstrxmnzlcwry.jpg?v=1722640536'),
 Document(id='ba1114c7-45f3-45f0-ac60-05b2e1f4f499', metadat

In [ ]:
# re.sub(r'[^a-zA-Z0-9\s]', '',
contexto = ". ".join([re.sub(r'\b\w+:\s*', '', doc.page_content) for doc in docs_relevantes])


In [ ]:
print(formatar_resposta(contexto))


Vinho Tinto Simis Cabernet Sauvignon Gran Reserva 2019 Cabernet Sauvignon robusto e complexo, com notas de cereja, figo e tabaco, ideal para carnes intensas e queijos fortes.
130.56 //cdn.dooca.store/158171/products/qo1mrdylixddysxzn2t3hvrvstrxmnzlcwry.jpg?v=1722640536.
﻿Vinho Tinto Simis Cabernet Sauvignon Gran Reserva 2019 Cabernet Sauvignon robusto e complexo, com notas de cereja, figo e tabaco, ideal para carnes intensas e queijos fortes.
130.56 //cdn.dooca.store/158171/products/qo1mrdylixddysxzn2t3hvrvstrxmnzlcwry.jpg?v=1722640536.
//www.vinicolagoes.com.br/vinho-tinto-sauv-cabernet-sauvignon-seco-269ml Vinho Tinto Sauv Cabernet Sauvignon Seco 269ml R$ 235,00 235.0 6950907 O Vinho Tinto Sauv Cabernet Sauvignon Seco &eacute; elaborado exclusivamente a partir de uvas 100% Cabernet Sauvignon, resultando em uma bebida com colora&ccedil;&atilde;o vermelho intenso e reflexos violetas.
No aroma, destaca-se o perfil frutado, com notas sutis de cereja e morango.
Na boca, apresenta corpo m&

In [ ]:
prompt = f"""
<role>
{role}
</role>
<contexto>:
{contexto}
</contexto>
<pergunta>:
{pergunta}
</pergunta>
<resposta>:
...
"""




In [ ]:
print(prompt)



<role>
Você é um sommelier experiente.
          trabalha em um renomado restaurante 5 estrelas
          Sua função é recomendar vinhos assertivos ao cliente
          voce acabou de receber uma descrição de vinho, e deve usar o contexto para construir uma recomendação apropriada,
          lembre que voce esta falando diretamente com o cliente, entao seja educado e profissional,
          tambem explique o porque o vinho escolhido é o melhor para o cliente, e como ele orna com o prato
</role>
<contexto>:
﻿Vinho Tinto Simis Cabernet Sauvignon Gran Reserva 2019 Cabernet Sauvignon robusto e complexo, com notas de cereja, figo e tabaco, ideal para carnes intensas e queijos fortes. 130.56 //cdn.dooca.store/158171/products/qo1mrdylixddysxzn2t3hvrvstrxmnzlcwry.jpg?v=1722640536. ﻿Vinho Tinto Simis Cabernet Sauvignon Gran Reserva 2019 Cabernet Sauvignon robusto e complexo, com notas de cereja, figo e tabaco, ideal para carnes intensas e queijos fortes. 130.56 //cdn.dooca.store/158171/product

In [ ]:
# wo_rag = get_answer(pergunta)


In [ ]:
with_rag = get_answer(prompt)


In [ ]:
# print("\nPergunta:")
# print(pergunta)

# print("\n====================\n")

# print("\nResposta sem RAG:")
# print(formatar_resposta(wo_rag))

# print("\n====================\n")

# print("Resposta com RAG:")
print(formatar_resposta(with_rag))


Para o prato tradicional e intensamente saboroso do Boeuf Bourguignon, eu recomendo a nossa Vinho Tinto Simis Cabernet Sauvignon Gran Reserva 2019.
Essa bebida é uma opção robusta e complexa que contém notas de cereja e figo combinadas com um sutil tom de tabaco, além de sua natureza seco, o qual se encontra ideal para carnes vermelhas fortes como o seu prato.
Essa combinação de características oferece uma sensação equilibrada e saborosa ao longo da experiência, acompanhando as notas intensas do Boeuf Bourguignon em cada gosto.
Além disso, sua cor vermelho intenso reflexa violetas se encontra harmoniosamente na composição deste prato tradicional.
Você pode encontrar nossa Vinho Tinto Simis Cabernet Sauvignon Gran Reserva 2019 nosso loja ou aqui no restaurante, e eu estou à sua disposição para qualquer outra recomendação de vinho que possamos oferecer.
